In [1]:
from scipy.stats import binom
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import scipy.stats as stats
import pymc3 as pm
import plotly.figure_factory as ff


### R code 2.1

In [2]:
ways = np.array([0,3,8,9,0])
ways/sum(ways)

array([ 0.  ,  0.15,  0.4 ,  0.45,  0.  ])

#### R code 2.2

In [3]:
binom.pmf(6,9,0.5)

0.16406250000000006

#### R code 2.3

In [4]:
p_grid = np.linspace(start =0, stop = 1, num=20)
prior = np.repeat(1,20)
# compute likelihood at each value in grid
likelihood = binom.pmf(6,9,p_grid)
# compute product of likelihood and prior
unstd_posterior = likelihood * prior

# standardize the posterior, so it sums to 1
posterior = unstd_posterior / sum(unstd_posterior)

# Create a trace
trace1 = go.Scatter(
    x = p_grid,
    y = posterior,
    mode = 'lines+markers'
)

p_grid = np.linspace(start =0, stop = 1, num=20)
prior = np.where(p_grid < 0.5, 0,1)



# compute likelihood at each value in grid
likelihood = binom.pmf(6,9,p_grid)
# compute product of likelihood and prior
unstd_posterior = likelihood * prior

# standardize the posterior, so it sums to 1
posterior = unstd_posterior / sum(unstd_posterior)

# Create a trace
trace2 = go.Scatter(
    x = p_grid,
    y = posterior,
    mode = 'lines+markers'
)



p_grid = np.linspace(start =0, stop = 1, num=20)
prior = np.exp(-5*abs(p_grid -0.5))



# compute likelihood at each value in grid
likelihood = binom.pmf(6,9,p_grid)
# compute product of likelihood and prior
unstd_posterior = likelihood * prior

# standardize the posterior, so it sums to 1
posterior = unstd_posterior / sum(unstd_posterior)

# Create a trace
trace3 = go.Scatter(
    x = p_grid,
    y = posterior,
    mode = 'lines+markers'
)


fig = tools.make_subplots(rows=1, cols=3, subplot_titles=('Prior = 1', 'Prior = ifelse( p_grid < 0.5 , 0 , 1 )',
                                                          'Prior = exp(-5*abs( p_grid - 0.5 ))'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)

py.iplot(fig, filename='make-subplots-multiple-with-titles')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]



### Code 2.6

In [8]:
data = np.repeat((0, 1), (12, 24))
with pm.Model() as normal_aproximation:
    p = pm.Uniform('p', 0, 1, transform=None)
    w = pm.Binomial('w', n=len(data), p=p, observed=data.sum())
    mean_q = pm.find_MAP()
    std_q = ((1/pm.find_hessian(mean_q))**0.5)[0]
print(mean_q['p'], std_q)

norm = stats.norm(mean_q, std_q)
prob = .89
z = stats.norm.ppf([(1-prob)/2, (1+prob)/2])
pi = mean_q['p'] + std_q * z 
print(pi)

/Users/sushmitroy/anaconda/lib/python3.6/site-packages/theano/tensor/basic.py:2146: UserWarning:

theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.



Optimization terminated successfully.
         Current function value: 1.966758
         Iterations: 6
         Function evaluations: 12
         Gradient evaluations: 10
0.6666666666399004 [ 0.07856742]
[ 0.54110075  0.79223258]


### Code 2.7

In [11]:
w = 24
n = 36
x = np.linspace(0, 1, 100)
ana_soln=stats.beta.pdf(x , w+1, n-w+1)

trace1 = go.Scatter(
    x = x,
    y = ana_soln,
    mode = 'line',
    name = 'analytical solution'
)


x = np.linspace(0, 1, 100)
quad_soln = stats.norm.pdf(x,loc = 0.67,scale = 0.07856742) 

trace2 = go.Scatter(
    x = x,
    y = quad_soln,
    mode = 'line',
    name = 'quadratic approximation'
)
layout = dict(title = 'n=36')
data_to_plot =[trace1,trace2]
# py.iplot(data_to_plot)

fig= go.Figure(data=data_to_plot, layout=layout)
py.iplot(fig)
